# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [2]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    load_dotenv("/content/drive/MyDrive/.env")
    from google.colab import drive

    drive.mount("/content/drive")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 62ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [3]:
from agentics.core.llm_connections import get_llm_provider
from crewai.tools import tool
from ddgs import DDGS

## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    

2025-09-09 11:33:36.214 | DEBUG    | agentics.core.llm_connections:<module>:90 - AGENTICS is connecting to the following LLM API providers:
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:93 - 0 - WatsonX
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:98 - 1 - Gemini
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:102 - 2 - OpenAI
2025-09-09 11:33:36.215 | DEBUG    | agentics.core.llm_connections:<module>:104 - Please add API keys in .env file to add or disconnect providers.
2025-09-09 11:33:36.224 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-09 11:33:36.224 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'


## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [ ]:
from crewai import Agent
doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=get_llm_provider()
)

## Define a Task

In [ ]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [ ]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [ ]:
result = crew.kickoff(inputs={"question": input("Ask your question> ") })

## Display the results as markdown

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

## Define a conversational agent

In [ ]:
from crewai import Agent, Task, Crew
search_assistant = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=False,
    verbose=True,
    llm=get_llm_provider()
)

task = Task(
    description="""Your task is to help the user performing his / her web search on a deep subject.  
     You will start by asking the user what is his / her topic of interest. Then you will ask follow up question
     to clarity the user needs while executing your web search and provideing intermediate results to the user.
     When you think you have enough information you will provide your final output""",
    expected_output="""A markdown document that summarize first the user requests, then provide a detailed answers and finally report supporting evidence from web search""",
    agent=search_assistant,
    human_input=True,
    markdown=True
)

crew = Crew(
    agents=[search_assistant],
    tasks=[task],
    verbose=False,
)
result = crew.kickoff(inputs={})

2025-09-09 11:33:55.673 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-09 11:33:55.673 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
